## Week 2

*Welcome back! First run all cells in Week 2 and 3 before scrolling down to week 4.*

We first download the required software: LangChain and its dependency `pypdf`

In [ ]:
!wget https://github.com/srhm-ca/generative-ai-for-journalists/raw/main/Data.zip
!unzip Data.zip

In [ ]:
!apt install python3.10-venv

In [ ]:
!source tutorial-env/bin/activate

In [ ]:
!pip install --upgrade pip
!pip install --upgrade langchain pypdf

We then load LangChain's `pypdf` loader.

In [ ]:
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

Let's first load our PDF...

In [ ]:
loader = PyPDFLoader("Data/2021-census-population-occupied-private-dwellings-community-2001-2021.pdf")

In [ ]:
single_pdf = loader.load_and_split()

In [ ]:
loader = PyPDFDirectoryLoader("Data/Supreme Court opinions 2014/")

In [ ]:
many_pdfs = loader.load_and_split()

To have the language model answer specific questions for each document in the Supreme Court folder, try out the code blocks below!

Having loaded both the single PDF and a directory of PDFs, let's now load the CSV.

In [ ]:
def llm(text):
  """
  This function takes a string as input and returns a string.

  Args:
    text: The input string.

  Returns:
    The output string.
  """

  return "This is the output string."

In [ ]:
for document in many_pdfs:
    llm(f"{document.page_content}\n\nDescribe the topic of this text in a single word.")

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

In [ ]:
loader = CSVLoader("Data/Urban_Design_and_Architecture_Awards_Recipients.csv")

In [ ]:
csv = loader.load()

Having loaded our data, we'll now download and load the embedding model.

In [ ]:
!pip install sentence_transformers > /dev/null

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

Let's try embedding some text. Observe the output. Once you've tried it, scroll down to continue.

In [ ]:
text = "This is a test document."

In [ ]:
embeddings.embed_query(text)

We now have a working embedding function. Let's install Chroma.

In [ ]:
!pip install -U chromadb

In [ ]:
from langchain.vectorstores import Chroma

Let's make a vector store for our loaded documents!

In [ ]:
db = Chroma.from_documents(single_pdf, embeddings)

In [ ]:
db.add_documents(csv)

In [ ]:
db.add_documents(many_pdfs)

Let's try retrieving a relevant document.

In [ ]:
query = "An award concerning art."
db.similarity_search(query)

In [ ]:
query = "What exceptions does Rule 606(b)(1) contain?"
db.similarity_search(query)

# Week 3

We'll now try making an agent. We'll start by downloading a library to let us run a local language model.

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python --upgrade --force-reinstall --no-cache-dir

Let's load the software.

In [ ]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

With that loaded, let's download a language model.

In [ ]:
!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_0.gguf

Okay! Let's try loading it.

In [ ]:
!pip install llama-cpp-python

In [ ]:
llm = LlamaCpp(
    model_path="mistral-7b-instruct-v0.1.Q4_0.gguf",
    temperature=0.8,
    max_tokens=200,
    n_ctx=4096,
    top_p=1,
    n_gpu_layers=-1,
    f16_kv=True,
    verbose=True,
    callback_manager=callback_manager
)

Let's try it out!

In [ ]:
llm("Where is Tokyo?")

Having verified the language model works, let's try making a tool for it to access our vector store.

In [ ]:
from langchain.chains import RetrievalQA

Let's define our agent.

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.chains import LLMMathChain
from langchain.utilities import SerpAPIWrapper
from langchain.agents.agent_toolkits import create_retriever_tool

In [ ]:
retriever = db.as_retriever()

In [ ]:
tools = [
    create_retriever_tool(
        retriever,
        name="Search knowledge",
        description="Useful for when you need to answer a question. If the user asks a question concerning the Supreme Court or Hamilton, Ontario, find the answer to their question with this tool. Only use this tool once.",
    ),
]

In [ ]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

And let's try it out.

In [ ]:
agent.run("Who, in Hamilton, won an award for art?")

# Week 4

Welcome back! We'll now try making an interface for our agent. First, let's download Gradio.

In [ ]:
!pip install gradio==3.48.0

Next, let's define a `message` function. This will let our Gradio interface talk to our agent.

In [ ]:
import gradio as gr

history = []

def predict(message, history):
    response = agent.run("<s>[INST] " + message + " [/INST]")
    return response

With that done, let's try loading an interface in just a few lines of code.

In [ ]:
gr.ChatInterface(predict, theme="soft", title="My Fancy Chatbot").launch(share=True)

Super! It works. We now can talk to an agent hooked up to some data through a fancy website.